### **Importación de datos, librerías y definición de funciones**

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
misc = pd.read_csv("../data/segregated/misc_v2.csv")
device = pd.read_csv("../data/segregated/device_v2.csv")
geoNetwork = pd.read_csv("../data/segregated/geoNetwork_v2.csv")
totals = pd.read_csv("../data/segregated/totals_v2.csv")
trafficSource = pd.read_csv("../data/segregated/trafficSource_v2.csv")
# hitsFiles = glob.glob("*_hists.csv")
# for fl in hitsFiles:
  # hits = pd.read_csv("data/fl.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [46]:
# device = pd.read_csv("../data/segregated/device_v2_new.csv")
# device = device.sample(903653)
# device['device'].sample(3)
print("misc", misc.iloc[531706])
print("device", device.iloc[531706])
print("geoNetwork", geoNetwork.iloc[531706])
print("totals", totals.iloc[531706])
print("trafficSource", trafficSource.iloc[531706])
# device['device']
# device[device['device'] == 'NaN']
# device[device['device'].isna() == True]['device']

misc Unnamed: 1         NaN
channelGrouping    NaN
fullVisitorId      NaN
visitNumber        NaN
visitStartTime     NaT
Name: 531706, dtype: object
device device    NaN
Name: 531706, dtype: object
geoNetwork geoNetwork    NaN
Name: 531706, dtype: object
totals totals    NaN
Name: 531706, dtype: object
trafficSource trafficSource    NaN
Name: 531706, dtype: object


In [4]:
# device.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# device

In [5]:
print(misc.shape)
print(device.shape)
print(geoNetwork.shape)
print(totals.shape)
print(trafficSource.shape)

(1799982, 9)
(1799982, 4)
(1799982, 4)
(1799982, 4)
(1799982, 4)


In [6]:
misc.drop(columns=['Unnamed: 0', 'visitId'], axis=1, inplace=True)
device.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
geoNetwork.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
totals.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)
trafficSource.drop(columns=['Unnamed: 0', 'fullVisitorId', 'visitId'], axis=1, inplace=True)

In [7]:
print(misc.shape)
print(device.shape)
print(geoNetwork.shape)
print(totals.shape)
print(trafficSource.shape)

(1799982, 7)
(1799982, 1)
(1799982, 1)
(1799982, 1)
(1799982, 1)


In [8]:
# recursive flatten function from https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
# Call example:
  # flattenCol = [flatten_json(json.loads(d)) for d in df['trafficSource']]
  # dataFrameOfFlattenCol = pd.DataFrame(flattenCol)
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

  
# Call example:
  # df = extract_day(df, 'month')
def extract_date_info(df, parametro):
    if parametro == "date":
        df["date"] = [d.date() for d in df["visitStartTime"]]
    if parametro == "time":
        df["time"] = [d.time() for d in df["visitStartTime"]]
    if parametro == "day":
        df["day"] = df["visitStartTime"].dt.day
    if parametro == "month":
        df['month'] = df["visitStartTime"].dt.month
    if parametro == "weekday":
        df['weekday'] = df["visitStartTime"].dt.weekday
    if parametro == "weekofyear": 
        df['weekofyear'] = df["visitStartTime"].dt.weekofyear
    return df

numberExc = 0
def catch(func, handle=lambda e : e, *args, **kwargs):
    try:
        return func(*args, **kwargs)
    except Exception as e:
#         check how do those lines look like
#         remove that line from all of the dataframes
        global numberExc
        numberExc += 1
        return handle(e)


### Limpieza 


In [9]:
# De misc mantenemos:
  # fullVisitorId
  # visitStartTime
  # channelGrouping

misc["visitStartTime"] = pd.to_datetime(misc["visitStartTime"], unit='s')
misc.drop(['date', 'socialEngagementType'], axis=1, inplace=True)

df = misc.copy()

In [10]:
# pd.set_option('display.max_colwidth', 1000)
# device.iloc[0]

In [11]:
# De device mantenemos:
  # browser
  # deviceCategory
  # operatingSystem

deviceFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in device['device']]

devi = pd.DataFrame(deviceFlatten)

# quest: language?
devi.drop(columns=['', 'browserSize', 'browserVersion', 'flashVersion', 'language', 
    'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName',
    'mobileDeviceModel', 'mobileInputSelector', 'operatingSystemVersion',
    'screenColors','screenResolution'], inplace=True)
isMobile = {False : 0, True : 1}
browser = {'Chrome': 1, 'Safari': 2, 'Firefox': 3, 'Internet Explorer': 4, 'Edge': 5, 
    'Android Webview': 0, 'Safari (in-app)': 0, 'Opera Mini': 0, 'Opera': 0,
    'UC Browser': 0}
category = {'desktop': 1, 'mobile': 2, 'tablet': 3}
operatingSystem = {'Windows': 1, 'Macintosh': 2, 'Android': 3, 'iOS': 4,'Linux': 5,
    'Chrome OS': 6, '(not set)': 0, 'Windows Phone': 0, 'BlackBerry': 0, 'Samsung': 0}

#we map our data to its corresponding value
devi["isMobile"] = devi["isMobile"].map(isMobile)
devi["browser"] = devi["browser"].map(browser)
devi["deviceCategory"] = devi["deviceCategory"].map(category)
devi["operatingSystem"] = devi["operatingSystem"].map(operatingSystem)

devi.drop(columns=['isMobile'], inplace=True)
df = df.join(devi)

In [12]:
print ("There we", numberExc, "exceptions")
numberExc = 0

There we 91661 exceptions


In [13]:
devi.shape

(1799982, 3)

In [14]:
df.shape

(1799982, 8)

In [15]:
device['device'].shape

(1799982,)

In [16]:
# De geoNetwork mantenemos:
  # coutry
  # region (que contiene ciudad si no está disponible)
  # city

geoNetworkFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in geoNetwork['geoNetwork']]
geo = pd.DataFrame(geoNetworkFlatten)
geo.drop(['latitude','longitude','networkLocation','cityId','continent','metro'], axis=1, inplace=True)

# QUEST: check that it works in the same way
# geo['region'].apply(lambda x: geo['country'] if x == '(not set)' else x)
# geo['region'].apply(lambda x: geo['country'] if x == 'not available in demo dataset' else x)

def region_country(geo):
    if geo['region'] == '(not set)':
        return geo['country']
    elif geo['region'] == 'not available in demo dataset':
        return geo['country']
    else:
        return geo['region']

geo['region']=geo.apply(region_country,axis=1)

df = df.join(geo)

In [17]:
print ("There we", numberExc, "exceptions")
numberExc = 0

There we 91661 exceptions


In [18]:
# De totals mantenemos:
  # hits
  # transactionRevenue
  # newVisits

#Transform json
totalsList = [flatten_json(catch(lambda: json.loads(d))) for d in totals['totals']]
totals_df = pd.DataFrame(totalsList)

#Remove visits column as it does not provide any information
totals_df = totals_df.drop(['', 'visits'], axis=1)

#Change nan in "newVisits" for 0. Binary, 1 it's new visit, 0 it's not. 
totals_df['newVisits'] = totals_df['newVisits'].fillna(0)

#Change nan in "transactionRevenue" for 0. 
totals_df['transactionRevenue'] = totals_df['transactionRevenue'].fillna(0)

#Change nan in "bounces" for 0
totals_df['bounces'] = totals_df['bounces'].fillna(0)

#There are some nan in pageviews but it is not trivial to find a value we can change these nan by logically.
#The corrlation with hits is 0.984, so we can keep hits that does not have any nan value. Keeping both could be redundant.
totals_df = totals_df.drop(['pageviews'], axis=1)

#Not 100% proven yet, but i have a big feeling that bounces is mainly 1 when there is only 1 hit by defenition.
#This means that even though the correlation doesn't provide a big value beetween them two, bounces is not giving much 
#additional info. Meaning using it is not relevant.
totals_df = totals_df.drop(['bounces'], axis=1)

# df.drop(['totals'], axis=1, inplace=True)
df = df.join(totals_df)
# df = pd.concat([df, totals_df])

In [19]:
print ("There we", numberExc, "exceptions")
numberExc = 0

There we 91661 exceptions


In [20]:
# De trafficSource mantenemos:
  # adHasContent
  # adPosition
  # campaign
  # isTrueDirect
  # medium
  # source

trafficFlatten = [flatten_json(catch(lambda: json.loads(d))) for d in trafficSource['trafficSource']]
traffic = pd.DataFrame(trafficFlatten)
# TO DO check campaignCode, it errored and I am not dropping it, but I want to drop it
traffic.drop(['', 'adwordsClickInfo_criteriaParameters', 'adwordsClickInfo_adNetworkType',
    'adwordsClickInfo_gclId', 'adwordsClickInfo_isVideoAd', 'adwordsClickInfo_page',
    'keyword', 'referralPath', 'campaignCode'], axis=1, inplace=True)

# adContent to bool adHasContent
traffic['adContent'] = traffic['adContent'].map(lambda x: 0 if str(x) == 'NoneType' else 1)

# rename to a simpler name
traffic.rename(index=str, columns={
    "adContent": "adHasContent",
    "adwordsClickInfo_slot": "adPosition"})

# set (none)s and (non set)s to NaNs
# This column will contain:
    # cpc -> Cost Per Click
    # affiliate -> Affiliate marketing is a type of performance-based marketing in which a business rewards one or more affiliates for each visitor or customer brought by the affiliate's own marketing efforts.
    # cpm -> Cost per impression
# entender qué implican estos valores. ¿cpc vienen de search y cpm de webs? ¿Se juntan? ¿Importan?
traffic.loc[traffic['medium'] == '(non set)','medium'] = np.nan
traffic.loc[traffic['medium'] == '(none)','medium'] = np.nan

# TO DO: traffic['source'] regex for the main domain (last whatevs.com), and drop less than 1000
# traffic['source'] = traffic['source'].str.extract('{"browser": "(.+?)"', expand=True)

df = df.join(traffic)

In [41]:
# traffic['medium'].

In [21]:
print ("There we", numberExc, "exceptions")
numberExc = 0

There we 91661 exceptions


In [22]:
# hits, not used as it won't be available in real use

In [23]:
list(df.columns.values)

['Unnamed: 1',
 'channelGrouping',
 'fullVisitorId',
 'visitNumber',
 'visitStartTime',
 'browser',
 'deviceCategory',
 'operatingSystem',
 '',
 'city',
 'country',
 'networkDomain',
 'region',
 'subContinent',
 'hits',
 'newVisits',
 'sessionQualityDim',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'adContent',
 'adwordsClickInfo_slot',
 'campaign',
 'isTrueDirect',
 'medium',
 'source']

In [34]:
dropList = list(df.filter(like='Unnamed',axis=1).columns.values)
# dropList.append("")
df.drop(columns=dropList, axis=1, inplace=True)

In [35]:
df.shape

(1799982, 25)

In [37]:
df.head(10)

,channelGrouping,fullVisitorId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,city,country,networkDomain,...,timeOnSite,totalTransactionRevenue,transactionRevenue,transactions,adContent,adwordsClickInfo_slot,campaign,isTrueDirect,medium,source
0,Organic Search,538928163114544921,1.0,2017-05-29 16:07:23,1.0,3.0,3.0,not available in demo dataset,United States,rr.com,...,178,NaN,0,NaN,1,NaN,(not set),NaN,organic,google
1,Organic Search,5863936343673668667,2.0,2017-05-29 19:43:31,1.0,1.0,1.0,Bengaluru,India,122-airtelbroadband.in,...,2289,NaN,0,NaN,1,NaN,(not set),True,organic,google
2,Organic Search,9993197829693068679,9.0,2017-05-29 23:09:51,2.0,2.0,4.0,San Jose,United States,comcast.net,...,340,NaN,0,NaN,1,NaN,(not set),True,organic,google
3,Referral,635666386021193101,13.0,2017-05-29 09:20:43,1.0,1.0,1.0,Osaka,Japan,i-revonet.jp,...,1880,NaN,0,NaN,1,NaN,(not set),True,referral,analytics.google.com
4,Organic Search,7772696018151280169,1.0,2017-05-29 10:19:06,1.0,1.0,1.0,not available in demo dataset,Spain,jazztel.es,...,560,NaN,0,NaN,1,NaN,(not set),NaN,organic,google
5,Referral,859852287808649256,1.0,2017-05-29 17:39:32,1.0,1.0,6.0,Cupertino,United States,comcast.net,...,207,NaN,0,NaN,1,NaN,(not set),NaN,NaN,(direct)
6,Affiliates,3602120395700633470,1.0,2017-05-29 09:20:44,4.0,1.0,1.0,Seoul,South Korea,unknown.unknown,...,167,NaN,0,NaN,1,NaN,Data Share Promo,NaN,affiliate,Partners
7,Direct,6634315572942911323,1.0,2017-05-29 15:11:00,1.0,1.0,1.0,not available in demo dataset,Thailand,3bb.co.th,...,212,NaN,0,NaN,1,NaN,(not set),True,NaN,(direct)
8,Organic Search,4199946232412788468,1.0,2017-05-30 04:20:15,1.0,1.0,1.0,not available in demo dataset,United States,(not set),...,962,69960000,68960000,1,1,NaN,(not set),NaN,organic,google
9,Organic Search,856922349860540280,1.0,2017-05-29 07:33:47,0.0,1.0,1.0,not available in demo dataset,Australia,optusnet.com.au,...,144,NaN,0,NaN,1,NaN,(not set),NaN,organic,google


In [27]:
df.to_csv("../data/train_v2_cleaned.csv")
df.to_pickle("../data/train_v2_cleaned.pkl")

### **Transformación**

### ** Minería de datos **

In [28]:
traffic

,adContent,adwordsClickInfo_slot,campaign,isTrueDirect,medium,source
0,1,NaN,(not set),NaN,organic,google
1,1,NaN,(not set),True,organic,google
2,1,NaN,(not set),True,organic,google
3,1,NaN,(not set),True,referral,analytics.google.com
4,1,NaN,(not set),NaN,organic,google
5,1,NaN,(not set),NaN,NaN,(direct)
6,1,NaN,Data Share Promo,NaN,affiliate,Partners
7,1,NaN,(not set),True,NaN,(direct)
8,1,NaN,(not set),NaN,organic,google
9,1,NaN,(not set),NaN,organic,google


### ** Interpretación y evaluación **

In [29]:
import numpy as np

In [30]:
mixed = pd.DataFrame(np.random.randn(6,4),columns=list('ABCD'))
mixed['A'].iloc[0] = 'A'

In [31]:
mixed['B'].dtype

dtype('float64')

In [32]:
loadedDF = pd.read_csv("../data/train_v2_cleaned.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3,5,9,10,11,12,13,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
loadedDF

,Unnamed: 0,Unnamed: 1,channelGrouping,fullVisitorId,visitNumber,visitStartTime,browser,deviceCategory,operatingSystem,city,...,timeOnSite,totalTransactionRevenue,transactionRevenue,transactions,adContent,adwordsClickInfo_slot,campaign,isTrueDirect,medium,source
0,0,NaN,Organic Search,538928163114544921,1.0,2017-05-29 16:07:23,1.0,3.0,3.0,not available in demo dataset,...,178.0,NaN,0,NaN,1,NaN,(not set),NaN,organic,google
1,1,NaN,Organic Search,5863936343673668667,2.0,2017-05-29 19:43:31,1.0,1.0,1.0,Bengaluru,...,2289.0,NaN,0,NaN,1,NaN,(not set),True,organic,google
2,2,NaN,Organic Search,9993197829693068679,9.0,2017-05-29 23:09:51,2.0,2.0,4.0,San Jose,...,340.0,NaN,0,NaN,1,NaN,(not set),True,organic,google
3,3,NaN,Referral,635666386021193101,13.0,2017-05-29 09:20:43,1.0,1.0,1.0,Osaka,...,1880.0,NaN,0,NaN,1,NaN,(not set),True,referral,analytics.google.com
4,4,NaN,Organic Search,7772696018151280169,1.0,2017-05-29 10:19:06,1.0,1.0,1.0,not available in demo dataset,...,560.0,NaN,0,NaN,1,NaN,(not set),NaN,organic,google
5,5,NaN,Referral,859852287808649256,1.0,2017-05-29 17:39:32,1.0,1.0,6.0,Cupertino,...,207.0,NaN,0,NaN,1,NaN,(not set),NaN,NaN,(direct)
6,6,NaN,Affiliates,3602120395700633470,1.0,2017-05-29 09:20:44,4.0,1.0,1.0,Seoul,...,167.0,NaN,0,NaN,1,NaN,Data Share Promo,NaN,affiliate,Partners
7,7,NaN,Direct,6634315572942911323,1.0,2017-05-29 15:11:00,1.0,1.0,1.0,not available in demo dataset,...,212.0,NaN,0,NaN,1,NaN,(not set),True,NaN,(direct)
8,8,NaN,Organic Search,4199946232412788468,1.0,2017-05-30 04:20:15,1.0,1.0,1.0,not available in demo dataset,...,962.0,69960000.0,68960000,1.0,1,NaN,(not set),NaN,organic,google
9,9,NaN,Organic Search,856922349860540280,1.0,2017-05-29 07:33:47,0.0,1.0,1.0,not available in demo dataset,...,144.0,NaN,0,NaN,1,NaN,(not set),NaN,organic,google
